In [1]:
## import modules to be used
import numpy as np
from util.data_preparation import VariedSizedImagesCollate, LoadingImgs_to_ListOfListOfTensors, VariedSizedImagesDataset, data_aug_preprocessing_HR
import time
import torch
from options.train_options import TrainOptions
# from data import create_dataset
from models import create_model
from util.visualizer import Visualizer
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from collections import OrderedDict
from util.metrics_utils import metrics_names
def linear_normalize(tmp):
    return (tmp - tmp.min())/(tmp.max() - tmp.min())

# if __name__ == '__main__':
name = '--name CUT+REG+Pseudo_9spl_4x_NCE20_REG100_TV600_MAE1_MAEpseudo10_3channel'
model = '--model cutreg_twostage' # current proposed framework that uses CUT loss and registration loss
# the switch for using plain CUT framework or our CUT+Registration framework
mode = '--train_R_with_G True --only_train_R False --only_train_G False --train_G_pseudo True'
hyperparameters = '--lambda_GAN 1.0 --lambda_NCE 20.0 --lambda_REG 100.0 --lambda_TV 600.0 --lambda_MAE 1.0 --lambda_MAE_pseudo 10.0 --CUT_mode FastCUT'
lr_schedule = '--n_epochs 50 --n_epochs_decay 450'
cmd_line = ' '.join((name, model, mode, hyperparameters, lr_schedule))
opt = TrainOptions(cmd_line).parse()   # get training options
model = create_model(opt)      # create a model given opt.model and other options
## loading the image data (full FOV, HR meaning original,non-downsampled)
data_folder = '../Data/Data_1212_22/'
good_image_index = [10, 11, 13, 14, 16, 22, 24, 25, 30]
list_of_list_of_tensors = LoadingImgs_to_ListOfListOfTensors(data_folder, good_image_index)
dataset = VariedSizedImagesDataset(list_of_list_of_tensors)
dataset_size = len(dataset) # get the number of images in the dataset.
train_dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_size=1, ## can only be 1
            collate_fn=VariedSizedImagesCollate,
            shuffle=not opt.serial_batches,
            num_workers=int(opt.num_threads),
            drop_last=True if opt.isTrain else False,
        )
test_dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, 
                                              sampler=torch.utils.data.RandomSampler(dataset, replacement=True, num_samples=2))
writer = SummaryWriter('runs/'+str(opt.name))

----------------- Options ---------------
                 CUT_mode: FastCUT                       
                LAB_space: False                         
               batch_size: 1                             
                    beta1: 0.5                           
                    beta2: 0.999                         
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: placeholder                   
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: None                          
            display_ncols: 4                             
             display_port: 8097                          
           display_server: htt

In [ ]:
# print('The number of training images = %d' % dataset_size)
# model.netG.load_state_dict(torch.load('checkpoints/CUT_newdata_512/160_net_G.pth'))
# model.netD.load_state_dict(torch.load('checkpoints/CUT_newdata_512/160_net_D.pth'))
# model.netG.load_state_dict(torch.load('checkpoints/CUTREG_NC6839_FFOV_cGAN/15_net_G.pth'))
model.netR.load_state_dict(torch.load('checkpoints/REG_35spl_4x_MAE_REG100TV600_ResUnet_3channel/500_net_R.pth'))
# model.netR.load_state_dict(torch.load('checkpoints/CUT+REG+Pseudo_9spl_4x_NCE20_REG100_TV600_MAE1_MAEpseudo10/30_net_R.pth'))
model.netG.load_state_dict(torch.load('checkpoints/'+ str(name)+ CUT+REG+Pseudo_9spl_4x_NCE20_REG100_TV600_MAE1_MAEpseudo10 +'/30_net_G.pth'))

visualizer = Visualizer(opt)   # create a visualizer that display/save images and plots
opt.visualizer = visualizer
total_iters = 0                # the total number of training iterations
optimize_time = 0.1
times = []
for epoch in range(opt.epoch_count, opt.n_epochs + opt.n_epochs_decay + 1):    # outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
    epoch_start_time = time.time()  # timer for entire epoch
    iter_data_time = time.time()    # timer for data loading per iteration
    epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch
    visualizer.reset()              # reset the visualizer: make sure it saves the results to HTML at least once every epoch

#     dataset.set_epoch(epoch)
    model.netG.train() # switching on the training mode for netG
#     model.netG.eval()
    running_losses = OrderedDict()
    for name in model.loss_names:
        running_losses[name] = 0
        
    for i, data in enumerate(train_dataloader): # inner loop within one epoch
        iter_start_time = time.time()  # timer for computation per iteration
        if total_iters % opt.print_freq == 0:
            t_data = iter_start_time - iter_data_time

        batch_size = len(data[0])
        total_iters += batch_size
        epoch_iter += batch_size

        if epoch == opt.epoch_count and i == 0:
            model.data_dependent_initialize(data) 
            model.setup(opt)               # regular setup: load and print networks; create schedulers
            model.parallelize()
            metrics = model.save_images_FFOV(0, test_dataloader)
            for i in range(len(metrics_names)):
                writer.add_scalar('metric_'+str(metrics_names[i]), metrics[i], epoch)
            
        ## stage 1: train image translation net with CUT loss with small FOV images, register first then translate
        ## netG, netF and netD is trainable, netR is fixed
        
        model.set_input(data)
        model.define_fold(data)
        model.forward_stage2()
#         real_A, real_B = model.real_A.detach().cpu().numpy(), model.real_B_reg.detach().cpu().numpy()
            
        # data_aug_preprocessing_HR crops large image into small patches
        mini_dataloader = model.patchify_WSI(real_B_reg = model.real_B_reg)       
        for i, mini_data in enumerate(mini_dataloader):
            if len(opt.gpu_ids) > 0:
                torch.cuda.synchronize()
            optimize_start_time = time.time()
            model.set_mini_input(mini_data)
            model.optimize_parameters()
            
            total_iters += model.mini_bs
            epoch_iter += model.mini_bs
            if len(opt.gpu_ids) > 0:
                torch.cuda.synchronize()
            optimize_time = (time.time() - optimize_start_time) / batch_size * 0.005 + 0.995 * optimize_time
            losses = model.get_current_losses()
            for name in model.loss_names:
                if isinstance(name, str):
                    running_losses[name] += losses[name] 
                
        ## stage 2: train image registration across full FOV, translate first then register
        ## netR is trainable, netG, netF, netD are fixed
        if not model.opt.only_train_G:
            model.set_input(data)
            fake_B = model.patch_wise_predict(input_data=model.real_A_eq, stride_ratio=1).permute(2,0,1).unsqueeze(0)
            
            if len(opt.gpu_ids) > 0:
                torch.cuda.synchronize()            
            optimize_start_time = time.time()
            model.set_input(data)
            ## gradient descend on netR
            model.optimize_parameters_stage2(fake_B = fake_B)
                
            model.netG.train()
            if len(opt.gpu_ids) > 0:
                torch.cuda.synchronize()
            optimize_time = (time.time() - optimize_start_time) / batch_size * 0.005 + 0.995 * optimize_time
            
        losses = model.get_current_losses()
        for name in model.loss_names:
            if isinstance(name, str):
                running_losses[name] += losses[name] 
        visualizer.print_current_losses(epoch, epoch_iter, losses, optimize_time, t_data)
        if opt.display_id is None or opt.display_id > 0:
            visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, losses)


#         if total_iters % opt.display_freq == 0:   # display images on visdom and save images to a HTML file
#             save_result = total_iters % opt.update_html_freq == 0
#             model.compute_visuals()
#             visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

        iter_data_time = time.time()

    if epoch % opt.save_epoch_freq == 0:              # cache our model every <save_epoch_freq> epochs
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
#         model.save_networks('latest')
#         model.save_networks(epoch)
        metrics = model.save_images_FFOV(epoch, test_dataloader)
        for i in range(len(metrics_names)):
            writer.add_scalar('metric_'+str(metrics_names[i]), metrics[i], epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' % (epoch, opt.n_epochs + opt.n_epochs_decay, time.time() - epoch_start_time))
    model.update_learning_rate()                     # update learning rates at the end of every epoch.
    for name in model.loss_names:
        if isinstance(name, str):
            writer.add_scalar('loss_'+str(name), running_losses[name]/len(train_dataloader), epoch)

Setting up a new session...


create web directory ./checkpoints/CUT+REG+Pseudo_9spl_4x_NCE20_REG100_TV600_MAE1_MAEpseudo10_3channel/web...


/projectnb/rise2019/OCT_CycleGAN/Torch_version/models/networks.py:571: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  patch_id = torch.tensor(patch_id, dtype=torch.long, device=feat.device)


---------- Networks initialized -------------
[Network G] Total number of parameters : 11.389 M
[Network F] Total number of parameters : 0.560 M
[Network D] Total number of parameters : 2.766 M
[Network R] Total number of parameters : 2.059 M
-----------------------------------------------
Directory Results/CUT+REG+Pseudo_9spl_4x_NCE20_REG100_TV600_MAE1_MAEpseudo10_3channel/images/ already exists


/share/pkg.7/pytorch/1.12.1/install/lib/SCC/../python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/projectnb/rise2019/OCT_CycleGAN/Torch_version/util/metrics_utils.py:30: RuntimeWarning: Mean of empty slice.
  L_mean, A_mean, B_mean = L[mask_WM].mean(), A[mask_WM].mean(), B[mask_WM].mean()
/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/numpy-1.19.5-py3.8-linux-x86_64.egg/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/numpy-1.19.5-py3.8-linux-x86_64.egg/numpy/lib/histograms.py:905: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/projectnb/rise2019/OC

...

(epoch: 86, iters: 25, time: 0.295, data: 0.002) G_GAN: 0.312 D_real: 0.087 D_fake: 0.353 G: 73.622 NCE: 73.246 REG: 4.513 TV: 0.389 MAE: 0.006 MAE_pseudo: 0.058 LAB: 0.000 
(epoch: 86, iters: 56, time: 0.295, data: 0.002) G_GAN: 0.638 D_real: 0.333 D_fake: 0.428 G: 94.354 NCE: 93.381 REG: 9.350 TV: 0.609 MAE: 0.031 MAE_pseudo: 0.304 LAB: 0.000 
(epoch: 86, iters: 81, time: 0.295, data: 0.002) G_GAN: 0.403 D_real: 0.465 D_fake: 0.135 G: 95.376 NCE: 94.710 REG: 6.771 TV: 0.822 MAE: 0.022 MAE_pseudo: 0.242 LAB: 0.000 
(epoch: 86, iters: 112, time: 0.295, data: 0.002) G_GAN: 0.291 D_real: 0.024 D_fake: 0.319 G: 84.013 NCE: 83.276 REG: 6.017 TV: 0.637 MAE: 0.031 MAE_pseudo: 0.416 LAB: 0.000 
(epoch: 86, iters: 137, time: 0.295, data: 0.002) G_GAN: 0.403 D_real: 0.221 D_fake: 0.145 G: 71.379 NCE: 70.788 REG: 6.137 TV: 0.979 MAE: 0.026 MAE_pseudo: 0.162 LAB: 0.000 
(epoch: 86, iters: 168, time: 0.295, data: 0.002) G_GAN: 0.385 D_real: 0.379 D_fake: 0.150 G: 79.647 NCE: 78.702 REG: 5.032 TV: 

/projectnb/rise2019/OCT_CycleGAN/Torch_version/models/networks.py:571: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  patch_id = torch.tensor(patch_id, dtype=torch.long, device=feat.device)


(epoch: 91, iters: 31, time: 0.295, data: 0.002) G_GAN: 0.735 D_real: 0.390 D_fake: 0.512 G: 82.663 NCE: 81.907 REG: 4.752 TV: 0.578 MAE: 0.002 MAE_pseudo: 0.020 LAB: 0.000 
(epoch: 91, iters: 62, time: 0.295, data: 0.002) G_GAN: 0.185 D_real: 0.085 D_fake: 0.507 G: 78.299 NCE: 78.023 REG: 5.063 TV: 0.616 MAE: 0.007 MAE_pseudo: 0.084 LAB: 0.000 
(epoch: 91, iters: 93, time: 0.295, data: 0.002) G_GAN: 0.312 D_real: 0.406 D_fake: 0.182 G: 80.198 NCE: 79.812 REG: 4.727 TV: 0.541 MAE: 0.009 MAE_pseudo: 0.065 LAB: 0.000 
(epoch: 91, iters: 118, time: 0.295, data: 0.002) G_GAN: 0.213 D_real: 0.123 D_fake: 0.411 G: 76.009 NCE: 75.738 REG: 4.446 TV: 0.399 MAE: 0.005 MAE_pseudo: 0.052 LAB: 0.000 
(epoch: 91, iters: 143, time: 0.295, data: 0.002) G_GAN: 0.278 D_real: 0.437 D_fake: 0.228 G: 71.073 NCE: 70.645 REG: 7.118 TV: 0.815 MAE: 0.025 MAE_pseudo: 0.125 LAB: 0.000 
(epoch: 91, iters: 168, time: 0.295, data: 0.002) G_GAN: 0.187 D_real: 0.138 D_fake: 0.402 G: 82.679 NCE: 82.406 REG: 6.618 TV: 

/projectnb/rise2019/OCT_CycleGAN/Torch_version/models/networks.py:571: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  patch_id = torch.tensor(patch_id, dtype=torch.long, device=feat.device)


(epoch: 96, iters: 25, time: 0.295, data: 0.001) G_GAN: 0.313 D_real: 0.137 D_fake: 0.267 G: 67.306 NCE: 66.919 REG: 5.289 TV: 1.043 MAE: 0.025 MAE_pseudo: 0.049 LAB: 0.000 
(epoch: 96, iters: 56, time: 0.295, data: 0.001) G_GAN: 0.322 D_real: 0.309 D_fake: 0.223 G: 81.491 NCE: 81.099 REG: 4.813 TV: 0.560 MAE: 0.008 MAE_pseudo: 0.062 LAB: 0.000 
(epoch: 96, iters: 81, time: 0.295, data: 0.001) G_GAN: 0.291 D_real: 0.269 D_fake: 0.225 G: 82.554 NCE: 82.186 REG: 6.314 TV: 0.787 MAE: 0.001 MAE_pseudo: 0.075 LAB: 0.000 
(epoch: 96, iters: 112, time: 0.295, data: 0.001) G_GAN: 0.921 D_real: 0.130 D_fake: 0.025 G: 98.391 NCE: 97.173 REG: 5.558 TV: 0.605 MAE: 0.027 MAE_pseudo: 0.270 LAB: 0.000 
(epoch: 96, iters: 137, time: 0.295, data: 0.001) G_GAN: 0.301 D_real: 0.200 D_fake: 0.222 G: 90.665 NCE: 90.078 REG: 5.151 TV: 0.972 MAE: 0.030 MAE_pseudo: 0.255 LAB: 0.000 
(epoch: 96, iters: 168, time: 0.295, data: 0.001) G_GAN: 0.220 D_real: 0.031 D_fake: 0.616 G: 82.945 NCE: 82.322 REG: 5.663 TV: 

0.19.3
